# Import Libraries

In [1]:
import pandas as pd
import random
from tqdm import tqdm

In [2]:
memes_df = pd.read_csv('warmemes.csv')

# VQA Dataset Preparation

In [3]:
questions = {
    'event_ua': 'Про яку подію йдеться у цьому мемі?',
    'organization_name': 'Про яку організацію йдеться у цьому мемі?',
    'person_name': 'Про яких людей чи персонажів йдеться у цьому мемі?',
    'location_name': 'Про які країни чи міста йдеться в цьому мемі?',
    'weapon_name': 'Про який вид зброї йдеться у цьому мемі?'
}

In [4]:
columns = ['image_path', 'question_type', 'question', 'answer']
memes_vqa_df = pd.DataFrame(columns=columns)

In [5]:
for question in questions.keys():
    filtered_memes_df = memes_df[memes_df[question].notna()][['image_path', question]]
    filtered_memes_df['question_type'] = question
    filtered_memes_df['question'] = questions[question]
    filtered_memes_df = filtered_memes_df.rename(columns={question: 'answer'})
    print(f'For question "{questions[question]}" added {len(filtered_memes_df)} rows')
    memes_vqa_df = pd.concat([memes_vqa_df, filtered_memes_df], ignore_index=True)

For question "Про яку подію йдеться у цьому мемі?" added 175 rows
For question "Про яку організацію йдеться у цьому мемі?" added 6 rows
For question "Про яких людей чи персонажів йдеться у цьому мемі?" added 44 rows
For question "Про які країни чи міста йдеться в цьому мемі?" added 51 rows
For question "Про який вид зброї йдеться у цьому мемі?" added 15 rows


In [6]:
memes_vqa_df

,image_path,question_type,question,answer
0,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,Путін оголосив війну Україні
1,images/war-and-peace.png,event_ua,Про яку подію йдеться у цьому мемі?,Російська влада заборонила називати російське ...
2,images/scale-of-ukrainian-heroism.jpg,event_ua,Про яку подію йдеться у цьому мемі?,"Через кілька днів стало зрозуміло, що Україна ..."
3,images/ghost-of-kyiv.png,event_ua,Про яку подію йдеться у цьому мемі?,Росія не змогла встановити перевагу в повітрі ...
4,images/sunflower-seeds.webp,event_ua,Про яку подію йдеться у цьому мемі?,Українка сказала російському солдату: «Поклади...
...,...,...,...,...
286,images/atacms.jpg,weapon_name,Про який вид зброї йдеться у цьому мемі?,ATACMS
287,images/727.jpg,weapon_name,Про який вид зброї йдеться у цьому мемі?,Кинджал
288,images/f-16-3.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,F-16
289,images/f-16-2.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,F-16


In [7]:
memes_vqa_df.to_csv("warmemes_vqa.csv", index=False)

# Multivariant VQA Dataset Preparation

In [8]:
multivariant_questions = {
    'event_ua': {
        'question': 'Про яку подію йдеться у цьому мемі?',
        'variants': list(memes_df.event_ua.dropna().unique())
    },
    'organization_name': {
        'question': 'Про яку організацію йдеться у цьому мемі?',
        'variants': list(memes_df.organization_name.dropna().unique())
    },
    'person_name': {
        'question': 'Про яких людей чи персонажів йдеться у цьому мемі?',
        'variants': list(memes_df.person_name.dropna().unique())
    },
    'location_name': {
        'question': 'Про які країни чи міста йдеться в цьому мемі?',
        'variants': list(memes_df.location_name.dropna().unique())
    },
    'weapon_name': {
        'question': 'Про який вид зброї йдеться у цьому мемі?',
        'variants': list(memes_df.weapon_name.dropna().unique())
    }
}

In [9]:
columns = ['image_path', 'question_type', 'question', 'variants', 'answer']
memes_mvqa_df = pd.DataFrame(columns=columns)

In [10]:
max_count = 10
variant_count = 4

for index, question in tqdm(memes_vqa_df.iterrows()):
    for _ in range(max_count):
        question = question.copy()
        all_variants = multivariant_questions[question['question_type']]['variants']
            
        variants = [question['answer']]
        for _ in range(1, variant_count):
            for i in range(10):
                candidate = random.choice(all_variants)
                if not(any(word in set(word for item in variants for word in item.split(';') if word) for word in candidate.split(';'))):
                    variants = variants + [candidate]
                    break
                
        if len(variants) == variant_count:
            random.shuffle(variants)
            question['variants'] = '\n'.join([f"{chr(97 + i)}) {value}" for i, value in enumerate(variants)])
            question_df = pd.DataFrame([question])
            memes_mvqa_df = pd.concat([memes_mvqa_df, question_df], ignore_index=True)

291it [00:00, 342.00it/s]


In [11]:
memes_mvqa_df

,image_path,question_type,question,variants,answer
0,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,"a) Однак Пригожин, мабуть, домовився з Путіним...",Путін оголосив війну Україні
1,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,a) У бою загинуло 250 тисяч росіян\nb) Путін о...,Путін оголосив війну Україні
2,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,a) Путін оголосив війну Україні\nb) Після звіл...,Путін оголосив війну Україні
3,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,a) Путін оголосив війну Україні\nb) У той же ч...,Путін оголосив війну Україні
4,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,a) Такер Карлсон взяв інтерв&#39;ю у Володимир...,Путін оголосив війну Україні
...,...,...,...,...,...
2898,images/f-16.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,a) F-16\nb) ATACMS\nc) Кинджал\nd) HIMARS,F-16
2899,images/f-16.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,a) ATACMS\nb) F-16\nc) HIMARS\nd) Кинджал,F-16
2900,images/f-16.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,a) ATACMS\nb) Кинджал\nc) HIMARS\nd) F-16,F-16
2901,images/f-16.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,a) F-16\nb) ATACMS\nc) HIMARS\nd) Кинджал,F-16


In [12]:
memes_mvqa_df = memes_mvqa_df.drop_duplicates()
memes_mvqa_df.reset_index(drop=True, inplace=True)
memes_mvqa_df

,image_path,question_type,question,variants,answer
0,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,"a) Однак Пригожин, мабуть, домовився з Путіним...",Путін оголосив війну Україні
1,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,a) У бою загинуло 250 тисяч росіян\nb) Путін о...,Путін оголосив війну Україні
2,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,a) Путін оголосив війну Україні\nb) Після звіл...,Путін оголосив війну Україні
3,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,a) Путін оголосив війну Україні\nb) У той же ч...,Путін оголосив війну Україні
4,images/war.jpg,event_ua,Про яку подію йдеться у цьому мемі?,a) Такер Карлсон взяв інтерв&#39;ю у Володимир...,Путін оголосив війну Україні
...,...,...,...,...,...
2861,images/f-16.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,a) ATACMS\nb) Кинджал\nc) HIMARS\nd) F-16,F-16
2862,images/f-16.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,a) F-16\nb) ATACMS\nc) HIMARS\nd) Кинджал,F-16
2863,images/f-16.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,a) HIMARS\nb) F-16\nc) ATACMS\nd) Кинджал,F-16
2864,images/f-16.webp,weapon_name,Про який вид зброї йдеться у цьому мемі?,a) F-16\nb) ATACMS\nc) Кинджал\nd) HIMARS,F-16


In [14]:
memes_mvqa_df.to_csv("warmemes_mvqa.csv", index=False)